#Лабораторная 3

## Импорты

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

In [85]:
df = pd.read_csv('/content/Econom_Cities_data.csv', sep=';', encoding='utf-8')

## Просмотр данных

In [86]:
df.shape

(48, 4)

In [87]:
list(df.columns)

['City', 'Work', 'Price', 'Salary']

In [88]:
df

,City,Work,Price,Salary
0,Amsterdam,1714,"65,6",49
1,Athens,1792,"53,8","30,4"
2,Bogota,2152,"37,9","11,5"
3,Bombay,2052,"30,3","5,3"
4,Brussels,1708,"73,8","50,5"
5,Buenos_Aires,1971,"56,1","12,5"
6,Cairo,-9999,"37,1",-9999
7,Caracas,2041,61,"10,9"
8,Chicago,1924,"73,9","61,9"
9,Copenhagen,1717,"91,3","62,9"


Проверяем из чего вообще состоят данные, есть ли нулы и какие типы данных

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    48 non-null     object
 1   Work    48 non-null     int64 
 2   Price   48 non-null     object
 3   Salary  48 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.6+ KB


In [90]:
df.describe()

,Work
count,48.000000
mean,1384.958333
std,2404.897007
min,-9999.000000
25%,1740.750000
50%,1834.500000
75%,1972.750000
max,2375.000000


##Чистка данных

Мне не понравилось, что price и  salary непонятного типа, приведем к числу

In [91]:
df["Price"]  = pd.to_numeric(df["Price"].astype(str).str.replace(",", "."))
df["Salary"] = pd.to_numeric(df["Salary"].astype(str).str.replace(",", "."))

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    48 non-null     object 
 1   Work    48 non-null     int64  
 2   Price   48 non-null     float64
 3   Salary  48 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.6+ KB


Уберем выбросы если есть по методу 3IQR

In [93]:
num_cols = ["Work","Price","Salary"]
num_df = df[num_cols].copy()

Q1 = num_df.quantile(0.25)
Q3 = num_df.quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 3 * IQR
upper = Q3 + 3 * IQR

mask = ((num_df >= lower) & (num_df <= upper)).all(axis=1)
removed_cities = df.loc[~mask, "City"].tolist()

df_clean = df.loc[mask].reset_index(drop=True)
num_clean = df_clean[num_cols].reset_index(drop=True)

print( removed_cities)

['Cairo', 'Jakarta']


##Кластеризация

Нормализуем данные

In [94]:
scaler = StandardScaler()
num_cols = ["Work", "Price", "Salary"]
X_scaled = scaler.fit_transform(df_clean[num_cols])
X = pd.DataFrame(X_scaled, columns=num_cols)

Для начала подберем параметры для модели DBSCAN

In [95]:
# берем значения по умолчанию
dbscan_1 = DBSCAN()

dbscan_1.fit(X)

dbscan_1.labels_

array([ 0, -1, -1, -1,  0, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1])

Посмотрим как часто встречается каждое из значений

In [96]:
table_ = pd.value_counts(pd.Series(dbscan_1.labels_))#преобразование матрицы в таблицу
table_.sort_index(inplace=True)

table_

/tmp/ipython-input-679088825.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  table_ = pd.value_counts(pd.Series(dbscan_1.labels_))#преобразование матрицы в таблицу


,count
-1,36
0,10


Получилось 36 выбросов....и 1 класс


Необходимо подобрать более качественые параметры

In [97]:
k = 2 #подбираем значение
for eps in [0.4, 0.5, 0.6, 0.8, 1.0, 1.2, 1.5]:
    db = DBSCAN(eps=eps, min_samples=k)
    labels = db.fit_predict(X_scaled)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    print(f"eps={eps:>4}: кластеры={n_clusters}, шумовых={n_noise}")

eps= 0.4: кластеры=8, шумовых=22
eps= 0.5: кластеры=6, шумовых=17
eps= 0.6: кластеры=5, шумовых=10
eps= 0.8: кластеры=4, шумовых=4
eps= 1.0: кластеры=3, шумовых=4
eps= 1.2: кластеры=2, шумовых=2
eps= 1.5: кластеры=1, шумовых=1


при eps= 0.8, а min_samples = 2, мы имеем самое адекватное кол-во выбросов(шумов) и кластеров

In [98]:
dbscan_2 = DBSCAN(eps=0.8, metric='euclidean', min_samples=2)

dbscan_2.fit(X)

unique, counts = np.unique(dbscan_2.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  4]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  2]]


Посмотрим как определились кластеры

In [99]:
df_clean['cluster'] = dbscan_2.labels_
df_clean

,City,Work,Price,Salary,cluster
0,Amsterdam,1714,65.6,49.0,0
1,Athens,1792,53.8,30.4,1
2,Bogota,2152,37.9,11.5,1
3,Bombay,2052,30.3,5.3,1
4,Brussels,1708,73.8,50.5,0
5,Buenos_Aires,1971,56.1,12.5,1
6,Caracas,2041,61.0,10.9,1
7,Chicago,1924,73.9,61.9,0
8,Copenhagen,1717,91.3,62.9,0
9,Dublin,1759,76.0,41.4,0


In [100]:
cluster_means = (
    df_clean.groupby("cluster")[["Work", "Price", "Salary"]]
    .mean()
    .round(3)
    .sort_index()
)
cluster_means

,Work,Price,Salary
cluster,,,
-1,2051.250,93.600,42.375
0,1792.000,77.526,55.158
1,1959.211,50.116,14.789
2,1874.000,97.950,95.150
3,1625.000,114.550,65.150


Мы выразили среднее значение признаков по кластерам, для более легкой интерпритации

* К **-1 кластеру** отнесли города аномалии или выбросы
* К **0му кластеру** подошли города с умеренными ценами и высокими зп
* К **1ему кластеру** относятся города с самой низкой зп и ценами, эти города экономически очень плохо развиты и ограницены.
* К **2 кластеру** относятся крупные города с высокими зп и ценами.
* К **3 кластеру** относятся города с ссамыми большими ценами, но не такими высокими зп, возможно какие-то туристически регионы

##Общий вывод



* DBSCAN: 4 кластера
* Иерархический анализ: 4 кластера
* Kmeans: 5 кластеров



* DBSCAN - показал себя лучше для такого набора данных, можно не чистить данные от выбрасов, сам определяет кол-во кластеров
* Иерархический анализ - подходит для разведочного анализа и понимания стуктуры файла, чувствителен к выбросам
* Kmeans - необходимо знать примерное кол-во кластеров для определения, самый простой из всех, понятная итерпритация результатов(от бедных к богатым)
